In [1]:
import findspark

findspark.init("/home/ozan/spark")

import os
os.environ["PYSPARK_SUBMIT_ARGS"]= '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

from pyspark.sql import SparkSession


In [3]:
spark=SparkSession.builder\
.config("spark.jars","/home/ozan/mysql-connector-java-8.0.27.jar")\
.master("local[4]")\
.appName("mysql")\
.getOrCreate()  

In [5]:
#mysql dataframe
mysqlDF=spark.read\
.format("jdbc")\
.option("driver","com.mysql.jdbc.Driver")\
.option("url","jdbc:mysql://127.0.0.1:3306/city")\
.option("dbtable","roads")\
.option("user","root")\
.option("password","1")\
.load()

In [6]:
mysqlDF.printSchema()

mysqlDF.show(truncate=False,n=5)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- capacity: integer (nullable = true)

+----+-----------------+--------+
|id  |name             |capacity|
+----+-----------------+--------+
|1001|34319. Sokak     |188     |
|1002|Dıbıl Sokak      |97      |
|1003|Sarayardı Caddesi|315     |
|1004|17091. Sokak     |158     |
|1005|25-1. Sokak      |188     |
+----+-----------------+--------+
only showing top 5 rows



In [7]:
kafkaDF=spark.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers","localhost:9092")\
.option("subscribe","road-sensor")\
.load()

In [8]:
from pyspark.sql.functions import from_json,when,sum,round
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [9]:
kafkaDF1=kafkaDF.selectExpr("CAST (value AS STRING)")

schema=StructType([StructField("id",StringType()),
                   StructField("cars",IntegerType())  
])

kafkaDF2=kafkaDF1.withColumn("value",from_json("value",schema))

#ouPut=kafkaDF2.writeStream.format("console").outputMode("append").start()

In [10]:
kafkaDF3=kafkaDF2.groupBy("value.id").agg(sum("value.cars").alias("total_cars"))

#ouPut=kafkaDF3.writeStream.format("console").outputMode("complete").start()

In [11]:
joinDF=mysqlDF.join(kafkaDF3,kafkaDF3.id==mysqlDF.id)

#ouPut=joinDF.writeStream.format("console").outputMode("complete").start()

In [12]:
joinDF1=joinDF.withColumn("rate",(round(joinDF.total_cars/joinDF.capacity,2)))

#ouPut=joinDF1.writeStream.format("console").outputMode("complete").start()

In [14]:
joinDF2=joinDF1.withColumn("road_condition",when(joinDF1.rate<0.25,"Yoğun Değil")\
                          .when((joinDF1.rate>=0.25)&(joinDF1.rate<0.50),"Az Yoğun")\
                          .when((joinDF1.rate>=0.50)&(joinDF1.rate<0.75),"Yoğun")\
                          .otherwise("Çok Yoğun"))

#ouPut=joinDF2.writeStream.format("console").outputMode("complete").start()

In [16]:
panelDF=joinDF2.select("rate","name","road_condition")

ouPut=panelDF.writeStream.format("console").outputMode("complete").start()

22/02/24 00:18:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-48f4c712-9852-4e7c-8ec8-972a3c670ea5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/02/24 00:18:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----+--------------+
|rate|name|road_condition|
+----+----+--------------+
+----+----+--------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+------------+--------------+
|rate|        name|road_condition|
+----+------------+--------------+
|0.02|1330/1 Sokak|   Yoğun Değil|
+----+------------+--------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----------------+--------------+
|rate|             name|road_condition|
+----+-----------------+--------------+
|0.02|     1330/1 Sokak|   Yoğun Değil|
|0.03|      25-1. Sokak|   Yoğun Değil|
|0.01|Sarayardı Caddesi|   Yoğun Değil|
|0.07|      Dıbıl Sokak|   Yoğun Değil|
+----+-----------------+--------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----------------+--------------+
|rate|             name|road_condition|
+----+-----------------+--------------+
|0.02|     1330/1 Sokak|   Yoğun Değil|
|0.03|      25-1. Sokak|   Yoğun Değil|
|0.03|Sarayardı Caddesi|   Yoğun Değil|
|0.07|      Dıbıl Sokak|   Yoğun Değil|
|0.02|     34319. Sokak|   Yoğun Değil|
+----+-----------------+--------------+



...

-------------------------------------------
Batch: 25
-------------------------------------------
+----+-----------------+--------------+
|rate|             name|road_condition|
+----+-----------------+--------------+
|0.25|     17091. Sokak|      Az Yoğun|
|1.44|     1330/1 Sokak|     Çok Yoğun|
|0.06|      25-1. Sokak|   Yoğun Değil|
|0.92|Sarayardı Caddesi|     Çok Yoğun|
| 1.3|      Dıbıl Sokak|     Çok Yoğun|
|0.16|     34319. Sokak|   Yoğun Değil|
+----+-----------------+--------------+



-------------------------------------------
Batch: 26
-------------------------------------------
+----+-----------------+--------------+
|rate|             name|road_condition|
+----+-----------------+--------------+
|0.25|     17091. Sokak|      Az Yoğun|
|1.44|     1330/1 Sokak|     Çok Yoğun|
|0.06|      25-1. Sokak|   Yoğun Değil|
|0.92|Sarayardı Caddesi|     Çok Yoğun|
|0.47|      Dıbıl Sokak|      Az Yoğun|
|0.16|     34319. Sokak|   Yoğun Değil|
+----+-----------------+--------------+



-------------------------------------------
Batch: 27
-------------------------------------------
+-----+-----------------+--------------+
| rate|             name|road_condition|
+-----+-----------------+--------------+
| 0.25|     17091. Sokak|      Az Yoğun|
|-0.02|     1330/1 Sokak|   Yoğun Değil|
| 0.06|      25-1. Sokak|   Yoğun Değil|
| 0.92|Sarayardı Caddesi|     Çok Yoğun|
| 0.47|      Dıbıl Sokak|      Az Yoğun|
| 0.16|     34319. Sokak|   Yoğun Değil|
+-----+-----------------+--------------+



-------------------------------------------
Batch: 28
-------------------------------------------
+----+-----------------+--------------+
|rate|             name|road_condition|
+----+-----------------+--------------+
|0.25|     17091. Sokak|      Az Yoğun|
|0.35|     1330/1 Sokak|      Az Yoğun|
|0.06|      25-1. Sokak|   Yoğun Değil|
|0.92|Sarayardı Caddesi|     Çok Yoğun|
|0.47|      Dıbıl Sokak|      Az Yoğun|
|0.16|     34319. Sokak|   Yoğun Değil|
+----+-----------------+--------------+



-------------------------------------------
Batch: 29
-------------------------------------------
+----+-----------------+--------------+
|rate|             name|road_condition|
+----+-----------------+--------------+
|0.25|     17091. Sokak|      Az Yoğun|
|0.71|     1330/1 Sokak|         Yoğun|
|0.06|      25-1. Sokak|   Yoğun Değil|
|0.92|Sarayardı Caddesi|     Çok Yoğun|
|0.47|      Dıbıl Sokak|      Az Yoğun|
|0.16|     34319. Sokak|   Yoğun Değil|
+----+-----------------+--------------+



In [17]:
ouPut.stop()